<a href="https://colab.research.google.com/github/gladcolor/360projection/blob/master/Maryland_data_into_labels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install dbfread
!pip install pyproj

     |████████████████████████████████| 10.4MB 2.8MB/s 


In [0]:
import os
import re
import glob
import requests
from dbfread import DBF
from pyproj import CRS
from pyproj import Transformer

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Make the directory if it does not exist
# You can change the directory as you'd like
# !mkdir 'drive/My Drive/research'
# !mkdir 'drive/My Drive/research/data'
%cd 'drive/My Drive/research/data'

/content/drive/My Drive/research/data


Get the list of data files

In [0]:
download_page = "https://apps.planning.maryland.gov/Secure/REST/SharepointService1.svc/GetOpenData"
attachments = requests.get(download_page).json()
links = []
for attachment in attachments:
  if attachment["FILENAME"].endswith("MPV15"):
    links.append((attachment["FILELINK"], attachment["FILENAME"]))

Download all of the data files

In [0]:
for link, filename in links:
  print(f"downloading {filename}")
  os.system(f"wget -nc {link}?dl=1 -O {filename}.zip")

downloading ALLE_MPV15
downloading ANNE_MPV15
downloading BACO_MPV15
downloading BACI_MPV15
downloading CALV_MPV15
downloading CARO_MPV15
downloading CARR_MPV15
downloading CECI_MPV15
downloading CHAR_MPV15
downloading DORC_MPV15
downloading FRED_MPV15
downloading GARR_MPV15
downloading HARF_MPV15
downloading HOWA_MPV15
downloading KENT_MPV15
downloading MONT_MPV15
downloading PRIN_MPV15
downloading QUEE_MPV15
downloading SOME_MPV15
downloading STMA_MPV15
downloading TALB_MPV15
downloading WASH_MPV15
downloading WICO_MPV15
downloading WORC_MPV15


Unzip data files (this takes a long time)


In [0]:
!unzip \*.zip

In [0]:
%ls
# make sure all the files are here

Alle2015/       CARO_MPV15.zip  FRED_MPV15.zip  locations.csv   Talb2015/
ALLE_MPV15.zip  Carr2015/       Garr2015/       Mont2015/       TALB_MPV15.zip
Anne2015/       CARR_MPV15.zip  GARR_MPV15.zip  MONT_MPV15.zip  Wash2015/
ANNE_MPV15.zip  Ceci2015/       Harf2015/       Prin2015/       WASH_MPV15.zip
Baci2015/       CECI_MPV15.zip  HARF_MPV15.zip  PRIN_MPV15.zip  weather/
BACI_MPV15.zip  Char2015/       Howa2015/       Quee2015/       Wico2015/
Baco2015/       CHAR_MPV15.zip  HOWA_MPV15.zip  QUEE_MPV15.zip  WICO_MPV15.zip
BACO_MPV15.zip  Dorc2015/       Kent2015/       Some2015/       Worc2015/
Calv2015/       DORC_MPV15.zip  KENT_MPV15.zip  SOME_MPV15.zip  WORC_MPV15.zip
CALV_MPV15.zip  erros.csv       labels.csv      Stma2015/
Caro2015/       Fred2015/       labels-old.csv  STMA_MPV15.zip


In [0]:
data_files = glob.glob("*/ATDATA/DATABASE/*2015.dbf")

In [0]:
# Setup transformer
crs_102285 = CRS.from_proj4("+proj=lcc +lat_1=38.3 +lat_2=39.45 +lat_0=37.66666666666666 +lon_0=-77 +x_0=400000 +y_0=0 +ellps=GRS80 +units=m no_defs")
# This is obtained from http://epsg.io/102285
crs_4326 = CRS.from_epsg(4326)
transformer = Transformer.from_crs(crs_102285, crs_4326)

In [0]:
# !rm locations.csv errors.csv
output = open("labels.csv", "w")
# errors = open("erros.csv", "w")
output.write("#,ACCTID,class,label text\n")
# open and prepare filse

row_num = 1

building_types = {}

for data_file in data_files:
  print(f"Processing {data_file}")
  dbf = DBF(data_file, encoding='iso-8859-1') # 
  for row in dbf:
    acctid = row["ACCTID"]
    story = row["DESCSTYL"]
    residential = row["RESIDENT"]
    lu = row["LU"]

    building_type = 0
    if lu in ["C", "EC"]: # commercial, exempt commercial
      building_type = 1
          
    story = row["BLDG_STORY"]
    story_num = 0
    if "STRY" in row["DESCSTYL"] and "INDUSTRY" not in row["DESCSTYL"]:
      story = row["DESCSTYL"].replace("Split Foyer", "2").lstrip(string.ascii_letters + " ").split(" Story")[0].replace(" 1/2", ".5")
      # FIXME: Add support for different types of residential buildings here. E.g. TH, SF, CN, AP, etc.
      story_num = min(int(float(story) * 2 - 1), 8)
    elif story != "":
      story_num = min(int(story.lstrip("0")) * 2 - 1, 8)
    else: # Unknown. Set everything to 0.
      building_type = 0
      story_num = 0
      

    class_num = building_type * 8 + story_num
    
    desc = ""

    building_types = ["residential", "commercial"]
    stories = ["Cannot tell", "1", "1.5", "2", "2.5", "3", "3.5", "4", "> 4"]

    if class_num != 0:
      desc += building_types[building_type]
      
    desc += stories[story_num]
      
    
    # print(acctid, story, residential)
    output.write(f"{row_num},{acctid},{class_num},{desc}\n")
    row_num += 1
  output.flush()

NameError: ignored